In [ ]:
import lyricsgenius

In [ ]:
client_access_token = '9srn9NndYAxR7oRSZ1TufUo8Z26uAI7z-ZZv27q-RB7izUz09WNI2l1b8Y9YMJZF'
genius = lyricsgenius.Genius(client_access_token, 
                             remove_section_headers=True, 
                             skip_non_songs=True, 
                             timeout=15) 

In [ ]:
from flask import Flask, request, jsonify
app = Flask(__name__)

In [ ]:
@app.route('/search', methods=['POST'])

def search_songs():
    data = request.json
    term = data.get("term", "")
    
    if not term:
        return jsonify({"error": "Title is required"}), 400
    
    try:
        response = genius.search_songs(term)
        hits = response.get("hits", [])
        
        songs = [
            {
                "title": hit["result"]["title"],
                "artist": hit["result"]["primary_artist"]["name"],
                "id": hit["result"]["id"]
            }
            for hit in hits if hit["type"] == "song"
        ]
        
        return jsonify({"songs": songs})
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [ ]:
@app.route('/lyrics', methods=['POST'])

def search_lyrics():
    data = request.json
    song_id = data.get("song_id", "")
    
    if not song_id:
        return jsonify({"error": "Song id is required"}), 400
    
    try:
        genius_lyrics = genius.lyrics(song_id)
        lyrics = genius_lyrics.splitlines()  
        trimmed_lyrics = "\n".join(lyrics[2:]) 
        
        return jsonify({"lyrics": trimmed_lyrics})
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [ ]:
if __name__ == "__main__":
    app.run(port=7000)